In [ ]:
# using Pkg
# Pkg.activate("../../Radial Duality/MultiCenterProject/MCRD_env")

In [ ]:
using LinearAlgebra
using Random, Distributions
using Printf
using Plots
using LaTeXStrings
Random.seed!(1234567) # Setting the seed

In [ ]:
# import MRM module
include("MRMs.jl")

In [ ]:
using JuMP
using MosekTools
using MathOptInterface

In [ ]:
function solveQCQPWithMosek(Ps, qs, rs, m, n)
    model = Model(Mosek.Optimizer)
    set_attribute(model, "QUIET", true)
    set_attribute(model, "INTPNT_CO_TOL_DFEAS", 1e-10)

    @variable(model, xvar[1:n]);

    for j in 2:m+1
        @constraint(model, rs[j] - qs[j]'*xvar - 0.5*xvar'*Ps[j]*xvar >= 0);
    end

    @objective(model, Max, rs[1] - qs[1]'*xvar - 0.5*xvar'*Ps[1]*xvar)
    optimize!(model)

    mosek_sol = JuMP.value.(xvar)
    mosek_obj_value = JuMP.objective_value(model)
    status = JuMP.termination_status(model)
    return mosek_sol, mosek_obj_value, status
end

# gauges for mosek solution
function getFeasVal(sol, qcqp, m)
    gauge_vals = Vector{Float64}(undef, m);
    for i in 1:m gauge_vals[i] = qcqp.identifiers[i](sol, false)[1] end;
    feasibility = maximum(gauge_vals)
    return feasibility
end;

In [ ]:
function quadratic(x, P, q, r)
    return r - q'*x - 0.5*x'*P*x
end
function getCentersAndRvals(m,numRvals,lower,upper,mats,vecs,cons,center_vecs,samp_mats,samp_rhos,samp_norms)
    Ps, qs, rs, best_centers = mats[m], vecs[m], cons[m], center_vecs[m]
    As, rhos, norms_of_Ps = samp_mats[m], samp_rhos[m], samp_norms[m]
    int_length = upper - lower
    
    # pre-allocations
    r_vals = Vector{Float64}(undef, numRvals)
    centers = Array{Float64}(undef, numRvals, m+1, n)
    all_rvals = Vector{Float64}(undef, m+1)
    for i = 1:numRvals
        u = int_length*rand(Uniform(0, 1)) + lower
        alpha_step = exp10(u)
        for j = 1:m+1
            bdry_point = MRM.sampleFromHilbertBallBoundary(As[j], best_centers[j], rhos[j], n)
            centers[i, j, :] = MRM.getNewCenter(bdry_point, Ps[j], norms_of_Ps[j], qs[j], alpha_step)
            all_rvals[j] = alpha_step * norm(Ps[j]*bdry_point + qs[j]) / norms_of_Ps[j]
            
            # checks
            val1 = quadratic(bdry_point, Ps[j], qs[j], rs[j])
            if abs(val1) >= tol
                @printf("Boundary point for quadratic %d has function value %.9f. \n", j, val1)
            end
            val2 = quadratic(centers[i, j, :], Ps[j], qs[j], rs[j])
            if val2 <= tol
                @printf("New center for quadratic %d has function value %.9f. \n", j, val2)
            end
        end
        r_vals[i] = minimum(all_rvals)
    end
    return centers, r_vals
end

function getGaps(fom_iter_pair, m, samp_centers, dict_r_vals, x0, tau0s, mosek_vals)
    centers, numRvals = samp_centers[m], length(dict_r_vals[m])
    tau0 = tau0s[m]
    mosek_opt = mosek_vals[m]
    fom, tot_iter = fom_iter_pair
    instance_constructor = MRM.makeQCQP_sqGauges
    if fom == MRM.subgradientMethod
        instance_constructor = MRM.makeQCQP
    end
    # preallocations
    gaps = Vector{Float64}(undef, numRvals)
    # solve problem for all values of R
    for i=1:numRvals
        current_centers = Array{Vector{Float64}}(undef, m+1)
        for j = 1:m+1
            current_centers[j] = centers[i, j, :]
        end
        inst = instance_constructor(mats[m], vecs[m], cons[m], current_centers, m, n, tol)
        # solve the problem 
        sol,_,_,_,_ = MRM.parallelMRM(fom,inst,(x0,tau0),tot_iter,b=b,N=N,tol=tol,rho_tilde=rho_tilde,tot_time=tot_time)
        val = 1 / sol.tau
        gaps[i] = mosek_opt - val
        if i % 50 == 0
            @printf("Solution value is %.6f for r-value number %d. \n", val, i)
            @printf("Gap to mosek val is %.6f. \n", gaps[i])
        end
    end
    return gaps
end;

In [ ]:
# problem params
tol = 10^(-9)
n = 200
lambda = 0.1
nu = 0.1
sigma = 10.0
x_sl = zeros(n)
x0 = x_sl;

In [ ]:
# pre-allocate dictionaries for storing values
m_vals = [10, 100, 1000]
K = length(m_vals)
mats = Dict{Int64, Array{Matrix{Float64}}}()
vecs = Dict{Int64, Array{Vector{Float64}}}()
cons = Dict{Int64, Array{Float64}}()
center_vecs = Dict{Int64, Array{Vector{Float64}}}()
f_x0s = Dict{Int64, Float64}()
tau0s = Dict{Int64, Float64}()

# sampling parameters
samp_mats = Dict{Int64, Array{Matrix{Float64}}}()
samp_rhos = Dict{Int64, Vector{Float64}}()
samp_norms = Dict{Int64, Vector{Float64}}()

# dictionary for centers and their R values
dict_r_vals = Dict{Int64, Vector{Float64}}()
samp_centers = Dict{Int64, Array{Float64}}()


# mosek stuff
mosek_sols = Dict{Int64, Vector{Float64}}()
mosek_vals = Dict{Int64, Float64}()

# dictionaries for each method
sub_gaps = Dict{Int64, Vector{Float64}}()
smooth_gaps = Dict{Int64, Vector{Float64}}()
genGrad_gaps = Dict{Int64, Vector{Float64}}();

In [ ]:
for k = 1:K
    m = m_vals[k]
    mats[m], vecs[m], cons[m], center_vecs[m] = MRM.sampleQCQP(n, m, lambda, sigma, nu)
    f_x0s[m] = quadratic(x0, mats[m][1], vecs[m][1], cons[m][1])
    tau0s[m] = 1 / f_x0s[m]
    # get sampling parameters
    samp_mats[m],samp_rhos[m],samp_norms[m] = MRM.getSamplingParameters(mats[m],vecs[m],cons[m],center_vecs[m])
end

In [ ]:
# solve with Mosek
for k = 1:K
    m = m_vals[k]
    mosek_sols[m], mosek_vals[m], _ = solveQCQPWithMosek(mats[m], vecs[m], cons[m], m, n)
end

In [ ]:
# parallel MRM parameters
N = 16
T_vals = [1, 1, 1]#[500, 100, 50]
rho_tilde = 1.0
tot_time = 90.0
b = 4.0;

In [ ]:
# experiment params
numRvals = 3#00
lower = -5.0
upper = 0.0;

In [ ]:
for k = 1:K
    m = m_vals[k]
    samp_centers[m],dict_r_vals[m] = getCentersAndRvals(m,numRvals,lower,upper,mats,vecs,cons,center_vecs,samp_mats,samp_rhos,samp_norms)
end

In [ ]:
# test if centers are indeed different
# for k=1:K
#     m = m_vals[k]
#     @printf("Printing values for m = %d \n", m)
#     for j = 1:m+1
#         vals = zeros(numRvals)
#         for i=1:numRvals
#             vals[i] = quadratic(samp_centers[m][i, j, :], mats[m][j], vecs[m][j], cons[m][j])
#         end
#         small = minimum(vals)
#         big = maximum(vals)
#         s_dev = std(vals)
#         mean_val = mean(vals)
#         @printf("Stats for j = %d are %.4f, %.4f, %.4f, %.4f  \n", j, small, mean_val, big, s_dev)
#     end
#     break
# end

In [ ]:
for k = 1:K
    m = m_vals[k]
    sub_pair = (MRM.subgradientMethod, T_vals[k])
    time_now = time()
    sub_gaps[m] = getGaps(sub_pair, m, samp_centers, dict_r_vals, x0, tau0s, mosek_vals)
    tot_sub_time = (time() - time_now) / 60;
    @printf("The total time for subgrad is %.4f minutes for m = %d. \n", tot_sub_time, m)
end

In [ ]:
for k = 1:K
    m = m_vals[k]
    smooth_pair = (MRM.fastSmoothingMethod, T_vals[k])
    time_now = time()
    smooth_gaps[m] = getGaps(smooth_pair, m, samp_centers, dict_r_vals, x0, tau0s, mosek_vals)
    tot_sm_time = (time() - time_now) / 60;
    @printf("The total time for smooth is %.4f minutes for m = %d. \n", tot_sm_time, m)
end

In [ ]:
for k = 1:2
    m = m_vals[k]
    genGrad_pair = (MRM.fastGenGradMethod, T_vals[k])
    time_now = time()
    genGrad_gaps[m] = getGaps(genGrad_pair, m, samp_centers, dict_r_vals, x0, tau0s, mosek_vals)
    tot_gen_time = (time() - time_now) / 60
    @printf("The total time for gengrad is %.4f minutes for m = %d. \n", tot_gen_time, m)
end

### Plots

In [ ]:
p = scatter()
fs = 14
ms = 3.5
for k = 1:3
    m = m_vals[k]
    init_gap = mosek_vals[m] - f_x0s[m]
    x = [1 / r for r in dict_r_vals[m]]
    yticks = [10.0^(-10+j) for j=1:10]
    p = scatter(x,sub_gaps[m]/init_gap, marker=(ms), xaxis=:log, yaxis=:log, legend=false)
    scatter!(x, smooth_gaps[m]/init_gap, xaxis=:log, yaxis=:log, marker=(ms), legend=false)
    if k < 3
        scatter!(x, genGrad_gaps[m]/init_gap, xaxis=:log, yaxis=:log, marker=(ms), legend=false)
    end
    ylims = ylims=(10.0^(-9), 1)
    yticks = [10.0^(-9+j) for j=1:9]
    yaxis!(yticks=yticks, ylims=ylims)
    xaxis!("Lipschitz constant 1 / R", xticks=[10^j for j=2:6])
    plot!(xtickfontsize=fs-2, ytickfontsize=fs-2)
    plot!(guidefontsize=fs, legendfontsize=fs)
    yaxis!("Relative Objective Gap")
    #savefig("../Plots/many_R_vals_m=$(m).pdf")
end
plot(p)